<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reviewa y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

## Librerías

In [59]:
pip install sweetviz 

Note: you may need to restart the kernel to use updated packages.


In [1]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import json
import os
import io
import ast
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import sweetviz as sv
import webbrowser

/opt/conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/opt/conda/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
/var/tmp/ipykernel_7326/3100989457.py:11: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_pr

<h2> Google Cloud Platform </h2>

In [2]:
# Google Cloud Platform
# Obtener el proyecto actual
project_id = os.environ.get("Proyecto Final - Henry")

# Crear un cliente de Cloud Storage
client = storage.Client(project=project_id)

# Obtener el nombre del bucket
bucket_name = "gmy"

# Obtener el objeto de referencia al bucket
bucket = client.bucket(bucket_name)

In [3]:
# Obtener el blob
blob = bucket.get_blob('etl/metadata-sitios-MISC.parquet')

# Descargar el blob
data = blob.download_as_string()

# Convertir el objeto de bytes a un objeto similar a un archivo
file = io.BytesIO(data)

# Leer el archivo parquet en un DataFrame
df = pd.read_parquet(file)

# Imprimir el DataFrame
df

,gmap_id,MISC
0,0x88f16e41928ff687:0x883dad4fd048e8f8,{'Accessibility': ['Wheelchair accessible entr...
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,{'Accessibility': ['Wheelchair accessible entr...
3,0x80c2c89923b27a41:0x32041559418d447,"{'Accessibility': None, 'Activities': None, 'A..."
4,0x80c2c632f933b073:0xc31785961fe826a6,"{'Accessibility': None, 'Activities': None, 'A..."
...,...,...
3025006,0x89c6c74f43a49b55:0x6be6995921c58b12,{'Accessibility': ['Wheelchair accessible entr...
3025007,0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,"{'Accessibility': None, 'Activities': None, 'A..."
3025008,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,"{'Accessibility': None, 'Activities': None, 'A..."
3025009,0x88d9e872a9e39f3d:0x5de29f50507e5d73,"{'Accessibility': None, 'Activities': None, 'A..."


In [4]:
df = df.dropna(subset=["MISC"])

In [5]:
df

,gmap_id,MISC
0,0x88f16e41928ff687:0x883dad4fd048e8f8,{'Accessibility': ['Wheelchair accessible entr...
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,{'Accessibility': ['Wheelchair accessible entr...
3,0x80c2c89923b27a41:0x32041559418d447,"{'Accessibility': None, 'Activities': None, 'A..."
4,0x80c2c632f933b073:0xc31785961fe826a6,"{'Accessibility': None, 'Activities': None, 'A..."
5,0x80c2cf163db6bc89:0x219484e2edbcfa41,{'Accessibility': ['Wheelchair accessible entr...
...,...,...
3025006,0x89c6c74f43a49b55:0x6be6995921c58b12,{'Accessibility': ['Wheelchair accessible entr...
3025007,0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,"{'Accessibility': None, 'Activities': None, 'A..."
3025008,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,"{'Accessibility': None, 'Activities': None, 'A..."
3025009,0x88d9e872a9e39f3d:0x5de29f50507e5d73,"{'Accessibility': None, 'Activities': None, 'A..."


In [13]:
df.iloc[2]["MISC"]

{'Accessibility': None,
 'Activities': None,
 'Amenities': None,
 'Atmosphere': None,
 'Crowd': None,
 'Dining options': None,
 'From the business': None,
 'Getting here': None,
 'Health & safety': None,
 'Health and safety': None,
 'Highlights': None,
 'Lodging options': None,
 'Offerings': None,
 'Payments': array(['Checks', 'Debit cards', 'Credit cards'], dtype=object),
 'Planning': None,
 'Popular for': None,
 'Recycling': None,
 'Service options': array(['In-store shopping'], dtype=object)}

In [34]:
MISC_dict_str = []
row = df.iloc[5]
MISC_dict = row["MISC"]
for service, description_list in MISC_dict.items():
    if description_list is not None:
        for description in description_list:
            MISC_dict_str.append({
                "gmapid": row["gmap_id"],
                "servicio": service,
                "descripción": description
            })
            print(row["gmap_id"],service,description)
# MISC_dict_str
    



0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Accessibility Wheelchair accessible entrance
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Amenities Good for kids
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Atmosphere Casual
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Crowd College students
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Dining options Lunch
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Dining options Dinner
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Health & safety Mask required
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Health & safety Staff wear masks
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Health & safety Staff required to disinfect surfaces between visits
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Offerings Comfort food
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Offerings Quick bite
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Offerings Small plates
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Popular for Lunch
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Popular for Dinner
0x80dd2b4c8555edb7:0xfc33d65c4bdbef42 Popular for Solo dining
0x80dd2b4c8555edb7

In [47]:
# Crear una lista de diccionarios con los datos del DataFrame
MISC_dict_str = []
for i in range(len(df)):
    row = df.iloc[i]
    MISC_dict = row["MISC"]
    for service, description_list in MISC_dict.items():
        if description_list is not None:
            for description in description_list:
                MISC_dict_str.append({
                    "gmapid": row["gmap_id"],
                    "servicio": service,
                    "descripción": description
                })

# Crear un DataFrame a partir de la lista de diccionarios
df_servicios = pd.DataFrame(MISC_dict_str)

# Eliminar duplicados
df_servicios = df_servicios.drop_duplicates(["gmapid", "servicio", "descripción"])


# Guardar el DataFrame en un CSV
df_servicios.to_csv(bucket.blob("etl/metadata-sitios-MISC-servicios.csv").open("w"), index=False)
# Guardar el DataFrame en un parquet
df_servicios.to_parquet(bucket.blob("etl/metadata-sitios-MISC-servicios.parquet").open("wb"))
 

In [50]:
df_servicios

,gmapid,servicio,descripción
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Accessibility,Wheelchair accessible entrance
1,0x88f16e41928ff687:0x883dad4fd048e8f8,Health & safety,Mask required
2,0x88f16e41928ff687:0x883dad4fd048e8f8,Health & safety,Staff required to disinfect surfaces between v...
3,0x88f16e41928ff687:0x883dad4fd048e8f8,Planning,Quick visit
4,0x88f16e41928ff687:0x883dad4fd048e8f8,Service options,In-store shopping
...,...,...,...
10657313,0x88d9e872a9e39f3d:0x5de29f50507e5d73,Health & safety,Mask required
10657314,0x88d9e872a9e39f3d:0x5de29f50507e5d73,Planning,Quick visit
10657315,0x88d9e872a9e39f3d:0x5de29f50507e5d73,Service options,Curbside pickup
10657316,0x88d9e872a9e39f3d:0x5de29f50507e5d73,Service options,In-store shopping


In [51]:
df_servicios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10593141 entries, 0 to 10657317
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   gmapid       object
 1   servicio     object
 2   descripción  object
dtypes: object(3)
memory usage: 323.3+ MB


In [52]:
df_servicios.describe()

,gmapid,servicio,descripción
count,10593141,10593141,10593141
unique,2315380,18,206
top,0x872b06f775659a61:0x65774b1aeb749bb1,Accessibility,Wheelchair accessible entrance
freq,64,2336421,1878590


In [70]:
# Generar el informe de análisis
profile = pandas_profiling.ProfileReport(df_servicios)

# Mostrar el informe
profile.to_file("metadata-sitios-MISC-servicios.html")
webbrowser.open("metadata-sitios-MISC-servicios.html")

# profile.to_file(bucket.blob("etl/metadata-sitios-MISC-servicios.html").open("w"))


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

False